Load model

Upload file

In [0]:
from google.colab import files
song = files.upload() 

Saving blues.00007.au to blues.00007.au


In [0]:
key = list(song.keys())[0]
key

'blues.00007.au'

In [0]:
import librosa
import pandas as pd
import numpy as np

import pathlib
import os

import timeit

Create dataframe

In [0]:
start = timeit.default_timer()

songname = key
y, sr = librosa.load(songname, mono=True, duration=5)

rmse = librosa.feature.rmse(y=y)
predict = []

# new features
flatness = librosa.feature.spectral_flatness(y=y)

S = np.abs(librosa.stft(y))
p0 = librosa.feature.poly_features(S=S, order=0)

tonnetz = librosa.feature.tonnetz(y=y, sr=sr)

chroma_cq = librosa.feature.chroma_cqt(y=y, sr=sr)

chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr) 

S = np.abs(librosa.stft(y))
contrast = librosa.feature.spectral_contrast(S=S, sr=sr)

y, sr = librosa.load(songname, mono=True, duration=30)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)

to_append = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)} {np.mean(flatness)} {np.mean(p0)} {np.mean(chroma_cq)} {np.mean(chroma_cens)} {np.mean(contrast)}'    
for e in mfcc:
    to_append += f' {np.mean(e)}'

for i in tonnetz:
    to_append += f' {np.mean(i)}'
#to_append += f' {g}'
#file = open('.csv', 'a', newline='')
#with file:
#    writer = csv.writer(file)
#    writer.writerow(to_append.split())
predict = to_append.split()
    
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  1.9895754239987582


In [0]:
from pandas import DataFrame

data = np.array(predict)
df = pd.DataFrame(data=data)
print(data)
print(data.shape)

['0.30792101611863837' '0.08432096242904663' '1451.75414666004'
 '1577.3699166309125' '2955.348796076819' '0.06143500266060371'
 '0.0012370450422167778' '0.6826974526763959' '0.48495920708894663'
 '0.26952969557106254' '21.721867047819373' '-179.39544684232592'
 '136.4592443182947' '-26.656358930559175' '39.98802679164439'
 '5.289679318941769' '10.924426906374084' '-20.561889109447083'
 '8.513764278793532' '-11.356908197621307' '-3.469077423708093'
 '-8.414554364140676' '-6.954826968424708' '-3.544535424055757'
 '-8.051242199794862' '-8.959536715733194' '-8.424337147616766'
 '-10.558884909220222' '-10.788158953621572' '-4.693748520349892'
 '-8.638613351936613' '-0.013363529370551712' '0.015988405395580548'
 '-0.06848223802241704' '-0.030022328910497012' '0.022893867199566067'
 '-0.009888853272147443']
(37,)


In [0]:
test = data.reshape((1, 1, 37))
test.shape

(1, 1, 37)

In [0]:
data_keras = data.reshape((1,37))
data_keras.shape

(1, 37)

In [0]:
genres_dict = {"0" : "blues", "1" : "classical","2" : "country", "3" : "disco","4" : "hiphop", "5" : "jazz","6" : "metal", "7" : "pop","8" : "reggae", "9" : "rock"}

Import ONNX model

In [0]:
!pip install onnx-tf

In [0]:
!pip install onnx

In [0]:
import onnx
import warnings
from onnx_tf.backend import prepare


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



/usr/local/lib/python3.6/dist-packages/onnx_tf/common/__init__.py:87: UserWarning: FrontendHandler.get_outputs_names is deprecated. It will be removed in future release.. Use node.outputs instead.
  warnings.warn(message)


In [0]:
# load onnx model
onnx_model_keras = onnx.load("model_keras.onnx")  
onnx_model_keras2 = onnx.load("model_keras2.onnx")
#onnx_model_lstm = onnx.load("model_lstm.onnx")
#onnx_model_bilstm = onnx.load("model_bilstm-onnx.onnx")

Make prediction

In [0]:
# run the loaded model
output_onnx_keras = prepare(onnx_model_keras) 
output_onnx_keras2 = prepare(onnx_model_keras2) 
#output_onnx_lstm = prepare(onnx_model_lstm) 
#output_onnx_bilstm = prepare(onnx_model_bilstm) 

#tf_rep = prepare(model) # Import the ONNX model to Tensorflow

/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op ConstantLike in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op DynamicSlice in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op ImageScaler in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:70: UserWarning: Fail to get since_version of ThresholdedRelu in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))


In [0]:
result_keras = output_onnx_keras.run(data_keras)
result_keras2 = output_onnx_keras2.run(data_keras)

predict_keras = np.argmax(result_keras)
predict_keras2 = np.argmax(result_keras2)


In [0]:
#print('Result for LSTM:',(result_lstm, genres_dict.get(str(predict_lstm))))
#print('Result for BiLSTM:',(result_bilstm, genres_dict.get(str(predict_bilstm))))
print('Result for Keras:',(result_keras, genres_dict.get(str(predict_keras))))
print('Result for Keras2:',(result_keras2, genres_dict.get(str(predict_keras2))))
print(songname)

Result for Keras: (Outputs(dense_4_Softmax_0=array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]], dtype=float32)), 'jazz')
Result for Keras2: (Outputs(dense_5_Softmax_0=array([[0.0000000e+00, 1.9758053e-28, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 9.9991405e-01, 0.0000000e+00, 8.5954824e-05,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)), 'jazz')
blues.00007.au


In [0]:
output_onnx_lstm = prepare(onnx_model_lstm)

RuntimeError: ignored

In [0]:
onnx_model_bilstm = onnx.load("model_bilstm4onnx.onnx")

In [0]:
output_onnx_bilstm = prepare(onnx_model_bilstm)

RuntimeError: ignored